## About

This agent randomly plays hands in for the first hundred steps and observes opponent's hands.  
After that, it predicts a next opponent's hand using last **_self's_** and opponent's hand.
  
Transition Matrix is used for predicton. Thanks to: https://www.kaggle.com/group16/rps-opponent-transition-matrix

In [ ]:
%%bash
pip install 'kaggle-environments>=0.1.6'

In [ ]:
%%writefile submission.py

import numpy as np
import pandas as pd
import random

T = np.zeros((3, 3, 3))

self_actions = np.full(1001, -1, dtype=int)
oppo_actions = np.full(1001, -1, dtype=int)

observe_until = 600

def observe_and_predict(observation, configuration):
    
    step = observation.step
    global T, P
    global self_actions, oppo_actions
    global observe_until
    
    if step == 0:
        self_act = np.random.randint(3)
        self_actions[step] = self_act
        return self_act
    
    self_1s_bef = self_actions[step - 1]
    oppo_1s_bef = observation.lastOpponentAction
    oppo_actions[step - 1] = oppo_1s_bef
    
    if 2 <= step < observe_until:
        self_2s_bef = self_actions[step - 2]
        oppo_2s_bef = oppo_actions[step - 2]
        T[self_2s_bef][oppo_2s_bef][oppo_1s_bef] += 1

    P = T / np.maximum(1, T.sum(axis=2)[..., None])    
    p = P[self_1s_bef][oppo_1s_bef]
    
    if observe_until <= step and np.sum(p) == 1:
        self_act = int((np.random.choice([0, 1, 2], p=p) + 1)) % 3
    else:
        self_act = np.random.randint(3)
    
    self_actions[step] = self_act
    return self_act

In [ ]:
%%writefile random_agent.py
import numpy as np
def random_agent(observation, configuration):
    return np.random.randint(3)

In [ ]:
%%writefile copy_opponent.py
def copy_opponent_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return 0

In [ ]:
%%writefile win_last_opponent.py
import numpy as np
def win_last_opponent (observation, configuration):
    if observation.step > 0:
        opp_hand = observation.lastOpponentAction
        return (opp_hand + 1) % 3
    else:
        return np.random.randint(3)

In [ ]:
from kaggle_environments import evaluate, make
env = make("rps", configuration={"episodeSteps": 1000})

In [ ]:
env.reset()
env.run(["submission.py", "copy_opponent.py"])
env.render(mode="ipython", width=800, height=800)

In [ ]:
env.reset()
env.run(["submission.py", "win_last_opponent.py"])
env.render(mode="ipython", width=800, height=800)

In [ ]:
env.reset()
env.run(["submission.py", "random_agent.py"])
env.render(mode="ipython", width=800, height=800)